## Robust Production - Inventory

The production inventory problem is one that addresses uncertainty in inventory management scenarios. The objective is to minimize the costs associated with inventory management while ensuring that customer demand is met consistently over a planning horizon. Since the demand is uncertain and varies within a specific bound, this problem can be classified as a robust optimization problem.

Takingn the example introduced in Ben-Tal et al. (2004), let us consider a single product inventory system which is comprised of a warehouse and $I$ factories. The planning horizon is $T$ periods. At period $t$:
- $d_t$ is the demand of the product that is uncertain
- $v(t)$ is the amount of product in the warehouse at the beginning of a period
- $p_i(t)$ is the amount of product to be produced during period $t$ by factory $i$ and is used to satisfy the demand of the period
- $P_i(t)$ is the maximal production capacity of factory $i$ in time period $t$
- $c_i(t)$ is the cost of producing a unit of product at factory $i$ in time period $t$
- $Q_i(t)$ is the maximal cumulative production capacity of factory $i$ in time period $t$
- $V_{min}$ and $V_{max}$ are the minimal and maximal storage capacity of the warehouse, respectively

With this information, the robust counterpart to the problem can be modeled into the following linear program:

$$
\begin{aligned}
& \text{minimize} \quad  F \\
& \text{subject to} \\
& \vec{c(t)}^T\vec{p(t)} \leq F,\\
& 0 \leq p_i(t) \leq P_i(t),\\
& 1^T\vec{p(t)} \leq Q_i,\\
& V_{min} \leq v(1) + 1^T\vec{p} - 1^T\vec{d} \leq V_{max}\\

\end{aligned}
$$


We first import the required packages and generate the data. 

In [1]:
import numpy as np 
import cvxpy as cp 
import lropt
import warnings
from scipy.sparse import SparseEfficiencyWarning
warnings.filterwarnings('ignore', category=UserWarning, module='cvxpy')
warnings.filterwarnings('ignore', category=SparseEfficiencyWarning)

Next, we can consider $I$ = 3 factories producing a product in one warehouse. The time horizon $T$ is 24 periods. The  maximal production capacity of each one of the factories at each two-weeks period is $P_i(t) = 567$, and the integral production capacity of each one of the factories for a year is $Q = 13600$. The inventory at the warehouse should not be less then $500$ units, and cannot exceed $2000$ units. Initially, the inventory sits at $500$ units.

In [2]:
np.random.seed(1)
T = 24 #number of periods
I = 3 #number of factories
V_MIN = 300 # The lower and upper bounds must be relaxed for larger uncertainty, or the problem becomes infeasible
V_MAX = 10000
HALF = 0.5
SEASONAL_MULTIPLIER = 1000
INTEGRAL_CAPACITY = 13600
RHO = 1
V_INITIAL = 500
MAX_CAPACITY = 567
PROPORTION = 0.2

P = np.full((I, T), MAX_CAPACITY)
Q = [INTEGRAL_CAPACITY]*I

F = cp.Variable(nonneg = True)
p = cp.Variable((I,T), nonneg = True)


The production cost for a factory $i$ at a period $t$ is given by:
$$
\begin{align*}
c_i(t) =  \alpha_i \left(1 - \frac{1}{2} \sin \left(\frac{\pi (t-1)}{12}\right)\right), \quad t  \in [1, 2, ... ,24] \\
\alpha_i \in [1, 1.5, 2]
\end{align*}
$$

In [3]:
c = [] 
alphas = np.array([1, 1.5, 2])

for t in range (1, T+1):
    c.append((alphas * (1 + HALF*np.sin(np.pi*(t-1)/(T*HALF)))).flatten())
c = np.array(c).T


In the robust optimization context, the demand is considered uncertain and seasonal, reaching its peak in winter. Specifically, the demand follows a seasonal pattern represented by:
$$
\begin{align*}
d^*(t) =  1000\left(1 - \frac{1}{2} \sin \left(\frac{\pi (t-1)}{12}\right)\right), \quad  t  \in [1, 2, ... ,24] \\
\end{align*}
$$


To handle this uncertainty, we adopt the norm uncertainty set. This uncertainty set is defined by:
$$
 \mathcal{U}_{\text{Norm}} = \{Az+b \ | \ z\| \|_p \le \rho\}
$$
This set allows us to define bounds for the uncertainty set to indicate that the demand is within an uncertainty level of $20\%$.

In [4]:
t_values = np.arange(1, T + 1)
d_star = SEASONAL_MULTIPLIER * (1 + HALF * np.sin(np.pi * (t_values - 1) / (T * HALF)))

# Construct lhs
eye_matrix = np.eye(T)
neg_eye_matrix = -np.eye(T)
lhs = np.concatenate((eye_matrix, neg_eye_matrix), axis=0)

# Construct rhs
rhs_upper = (1 + PROPORTION) * d_star
rhs_lower = (-1+PROPORTION) * d_star
rhs = np.hstack((rhs_upper, rhs_lower))

# Uncertain parameter definition
# d = lropt.UncertainParameter(T, uncertainty_set=lropt.Ellipsoidal(p=2,rho=5000, c=lhs, d=rhs, b = d_star )) #uncertain demand
d = lropt.UncertainParameter(T, uncertainty_set=lropt.Polyhedral(lhs=lhs, rhs=rhs)) #uncertain demand

Next, we will define all the constraints of the problem in a list.

In [5]:
constraints = [
    cp.sum(cp.multiply(c, p)) <= F,
    p <= P,
    cp.sum(p,1) <= Q,
]
for i in range(1,T+1):
    multvec = np.hstack([np.ones(i),np.zeros(T-i)])
    constraints += [cp.sum(cp.sum(p,0)[:i]) - multvec@d + V_INITIAL >= V_MIN]
    constraints += [cp.sum(cp.sum(p,0)[:i]) - multvec@d + V_INITIAL <= V_MAX]

Finally, we can define the objective and solve the problem. 

In [6]:
objective = cp.Minimize(F)
prob = lropt.RobustProblem(objective, constraints)
prob.solve()
sol = F.value
print(f"The robust optimal value is {sol}")
print(np.round(p.value))

The robust optimal value is 43670.829128431986
[[567. 567. 567. 567. 567. 567. 567. 567. 567. 567. 567. 567. 567. 567.
  567. 567. 567. 567. 567. 567. 567. 559. 567. 567.]
 [567. 567. 567. 567. 567. 567. 567. 567. 567. 567. 567. 567. 567. 478.
  333. 209. 113. 362. 567. 289.   0.   0.  33.  29.]
 [567. 567. 567. 567. 567. 363.   0. 307. 567. 490. 366. 221.  66.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]


In [7]:
for i in range(1,T+1):
  multvec = np.hstack([np.ones(i),np.zeros(T-i)])
  low = (cp.sum(cp.sum(p,0)[:i]) - multvec@rhs_upper + V_INITIAL).value
  mid = (cp.sum(cp.sum(p,0)[:i]) - multvec@d_star + V_INITIAL).value
  high = (cp.sum(cp.sum(p,0)[:i]) - multvec@(-rhs_lower) + V_INITIAL).value
  print(low, mid, high)

1000.9999999999577 1200.9999999999577 1400.9999999999577
1346.7085729379091 1772.5904774481614 2198.4723819584133
1547.7085729371033 2223.5904774473556 2899.4723819576075
1624.4445042238212 2571.037086852727 3517.629669481634
1605.8292619498825 2839.0243849572325 4072.2195079645844
1323.0985638359161 2852.886269472173 4382.673975108431
657.0985638447291 2486.886269480987 4316.673975117244
318.61524227457267 2444.995530539736 4571.3758188049
300.0000000006221 2712.9828286442316 5125.965657287839
300.00000000085856 2983.6935067631202 5667.387013525386
300.0000000006148 3233.6935067628765 6167.387013525142
300.0000000005384 3459.5754112730538 6619.150822545571
299.99999999983993 3659.5754112723553 7019.150822544872
300.00000000121145 3833.693506763473 7367.387013525738
300.0000000016262 3983.693506763888 7667.387013526153
300.00000000241926 4112.982828646029 7925.9656572896365
300.0000000041655 4226.380288269331 8152.760576534496
608.3736886593942 4638.16139429565 8667.94909993191
1142.37